In [ ]:
import pandas as pd
import numpy as np

### Dumb model
It is important to understand what is the random loss value, so that we can understand how good is our current best losses (around 0.0185) in leaderboard. 


### 0.5 for each class?
But providing 0.5 as the prediction for each class is not the dumb loss (it is dumber). This would make sense only if half of the test records would have each of the class as positive. But we have a highly unbalanced case here. We need to take the class prevalence into consideration. Let us try doing that.

### Class prevalence?

Consider the training set which has 23814 records. For each class find out how many times it has ones. Let us say N. We are going to set the probability as N/23814 for each test record for that class.

In [ ]:
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
train_targets_positive = train_targets_scored.sum()[1:]
train_targets_positive.sort_values(ascending=True)

As you see above, some of the classes are just having prevalences of 1, but this cannot happen as it was [made clear](https://www.kaggle.com/c/lish-moa/discussion/184005) that every class should have atleast 6 truth values. So let us look into each class and ensure that we have prevalences as multiples of 6.

In [ ]:
remainder = train_targets_positive % 6
number_to_be_added = 6 - remainder
number_to_be_added = number_to_be_added % 6
train_targets_positive_corrected = train_targets_positive + number_to_be_added

For each class, divide the corrected prevalence with total number of records. Here the correction is applied to total number of records as well.

In [ ]:
dumb_pred_for_each_class = train_targets_positive_corrected/(23814 + number_to_be_added.sum())

In [ ]:
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
submission.iloc[:, 1:] = dumb_pred_for_each_class.values

### Post Processing

In [ ]:
# https://www.kaggle.com/c/lish-moa/discussion/180165 
vehicle_indices = test_features[test_features["cp_type"]=="ctl_vehicle"].index.tolist()
submission.iloc[vehicle_indices, 1:] = np.zeros((1, 206))

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=False)

### Score

So with this dumb model, (in fact no model at all) I am able to get a score of **0.02359**. This is better that nearly 1/3 rd submissions in the leaderboard. 

So questions to ask
0.0185 does not seem to be very high compared to the dumb score of 0.02359. Why are we unable to get a really good score? Less data?
What is a good score in this compitition?